In [31]:
# notebooks/langchain.ipynb

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field, model_validator
from typing import List, Union

# Search queries response format
class SearchQuery(BaseModel):
    section: str = Field(..., description="The section of the outline the query corresponds to.")
    query: str = Field(..., description="The search query for this section.")


# Knowledgebase article format
class ReferenceItem(BaseModel):
    reference_number: int = Field(..., description="Sequential reference number.")
    authors: str = Field(..., description="Author(s) of the reference.")
    year: str = Field(..., description="Publication year.")
    title: str = Field(..., description="Title of the reference.")
    journal_source: str = Field(..., description="Journal or source of publication.")
    url_doi: str = Field(..., description="URL or DOI of the reference.")


class FAQItem(BaseModel):
    question: str = Field(..., description="A frequently asked question about the condition.")
    answer: str = Field(..., description="A concise and informative answer to the question.")


class Overview(BaseModel):
    heading: str = Field(default="Overview", description="Heading of the Overview section.")  # Corrected
    content: str = Field(..., description="A high-level introduction to the topic, explaining its significance and impact, key statistics, and facts.")


class KeyFacts(BaseModel):
    heading: str = Field(default="Key Facts", description="Heading of the Key Facts section.")  # Corrected
    content: List[str] = Field(..., description="Notable statistics or key data, presented as a list of strings.")


class Symptoms(BaseModel):
    heading: str = Field(default="Symptoms", description="Heading of the Symptoms section.")  # Corrected
    content: List[str] = Field(..., description="Common signs and symptoms, presented as a list of strings.")


class Types(BaseModel):
    heading: str = Field(default="Types", description="Heading of the Types section.")  # Corrected
    content: str = Field(..., description="Classifications or subtypes, each with a brief explanation. Types and subtypes may use nested subheadings (###, ####).")


class Causes(BaseModel):
    heading: str = Field(default="Causes", description="Heading of the Causes section.")  # Corrected
    content: str = Field(..., description="Explanation of underlying mechanisms or causes, including primary causes.")


class RiskFactors(BaseModel):
    heading: str = Field(default="Risk Factors", description="Heading of the Risk Factors section.")  # Corrected
    content: List[str] = Field(..., description="Factors that increase susceptibility, including lifestyle, genetic, or environmental risk factors.")


class Diagnosis(BaseModel):
    heading: str = Field(default="Diagnosis", description="Heading of the Diagnosis section.")  # Corrected
    content: str = Field(..., description="Outline of the diagnostic process, including medical history, clinical symptoms, tests, or imaging tools. Can include subheadings (###) for specific methods.")


class Prevention(BaseModel):
    heading: str = Field(default="Prevention", description="Heading of the Prevention section.")  # Corrected
    content: List[str] = Field(..., description="Practical advice for risk reduction, with evidence-based recommendations.")


class SpecialistToVisit(BaseModel):
    heading: str = Field(default="Specialist to Visit", description="Heading of the Specialist to Visit section.")  # Corrected
    content: str = Field(..., description="Healthcare providers involved in diagnosis and treatment, and their roles.")


class Treatment(BaseModel):
    heading: str = Field(default="Treatment", description="Heading of the Treatment section.")  # Corrected
    content: str = Field(..., description="Description of medical and therapeutic interventions.")


class HomeCare(BaseModel):
    heading: str = Field(default="Home-Care", description="Heading of the Home Care section.")  # Corrected
    content: List[str] = Field(..., description="Tips for self-management, such as lifestyle adjustments, routines, or home remedies.")


class LivingWith(BaseModel):
    heading: str = Field(default="Living With", description="Heading of the Living With section.")  # Corrected
    content: str = Field(..., description="Guidance for long-term management, including emotional, social, or physical adaptation strategies.")


class Complications(BaseModel):
    heading: str = Field(default="Complications", description="Heading of the Complications section.")  # Corrected
    content: str = Field(..., description="Discussion of potential health challenges if the condition is untreated or poorly managed.")


class AlternativeTherapies(BaseModel):
    heading: str = Field(default="Alternative Therapies", description="Heading of the Alternative Therapies section.")  # Corrected
    content: str = Field(..., description="Summary of non-conventional approaches.")


class FAQs(BaseModel):
    heading: str = Field(default="FAQs", description="Heading of the FAQs section.")  # Corrected
    content: List[FAQItem] = Field(..., description="A list of frequently asked questions and their corresponding answers.")


class References(BaseModel):
    heading: str = Field(default="References", description="Heading of the References section.")  # Corrected
    content: List[ReferenceItem] = Field(..., description="A list of references.")


class Outline(BaseModel):
    title: str = Field(..., description="The main heading of the article.")
    subtitle: str = Field(..., description="A concise introductory phrase summarizing the condition.")
    
    overview: Overview = Field(..., description="Overview section of the article.")
    key_facts: KeyFacts = Field(..., description="Key Facts section of the article.")
    symptoms: Symptoms = Field(..., description="Symptoms section of the article.")
    types: Types = Field(..., description="Types section of the article.")
    causes: Causes = Field(..., description="Causes section of the article.")
    risk_factors: RiskFactors = Field(..., description="Risk Factors section of the article.")
    diagnosis: Diagnosis = Field(..., description="Diagnosis section of the article.")
    prevention: Prevention = Field(..., description="Prevention section of the article.")
    specialist_to_visit: SpecialistToVisit = Field(..., description="Specialist to Visit section of the article.")
    treatment: Treatment = Field(..., description="Treatment section of the article.")
    home_care: HomeCare = Field(..., description="Home-Care section of the article.")
    living_with: LivingWith = Field(..., description="Living With section of the article.")
    complications: Complications = Field(..., description="Complications section of the article.")
    alternative_therapies: AlternativeTherapies = Field(..., description="Alternative Therapies section of the article.")
    faqs: FAQs = Field(..., description="FAQs section of the article.")
    references: References = Field(..., description="References section of the article.")


In [22]:
def get_prompt(condition, alternative_name, category):
        return f"""You are a professional scientific writer tasked with developing a detailed and informative knowledgebase article outline on a given condition.

    Condition: '{condition}'
    Alternate Name: '{alternative_name}'
    Category: '{category}'

    Detailed Instructions for Each Section:

    - **Title**: The main heading of the article, which is the condition itself (e.g., "Diabetes").

    - **Subtitle**: A concise introductory phrase summarizing the condition.

    - **Sections**: Each section in the article is structured as a key-value pair inside an array. Each element in the array must contain a `"heading"` and `"content"` entry.

        - **Heading**: A string providing the title of the heading. Ensure that all of these headings are included: `"Overview"`, `"Key Facts"`, `"Symptoms"`, `"Types"`, `"Causes"`, `"Risk Factors"`, `"Diagnosis"`, `"Prevention"`, `"Specialist to Visit"`, `"Treatment"`, `"Home-Care"`, `"Living With"`, `"Complications"`, `"Alternative Therapies"`, `"FAQs"`, `"References"`.

        - **Content**: The content for the associated heading. You must adhere to the following requirements:

            - Maintain a professional yet approachable tone.
            - Include hypertext links to relevant sources where appropriate, and format them in the `"References"`, in the same way as a traditional scientific paper.
            - Where possible, include statistics, research findings, or notable insights to make the article credible and informative.
            - Use bullet points where specified to ensure that information is captured in the output.
            - Ensure that content is well-written and contains a brief but sufficient summary.
            - Where subtypes exist, include nested subheadings within the `content` using the `###` markdown header format.

            - **FAQs**: The `content` for the `FAQs` section should be a JSON array of question-answer pairs. Each pair should have a "question" field and an "answer" field. The questions and answers should be concise, clear, and relevant to the condition. Generate 3 to 5 FAQs.

    ---

    Output Requirements:

    - Return only the refined JSON object.
    - Do not add any extraneous commentary, reasoning steps, or notes outside the JSON object.
    - Ensure that all integrated references are formatted in an APA-like style and included in the `"References"` section.
    """

In [27]:
import google.generativeai as genai

import json
import dataclasses
import typing_extensions as typing


model = genai.GenerativeModel(model_name="models/gemini-1.5-flash-latest")

prompt = get_prompt("Diabetes", "Hyperglycemia", "Metabolic Disorders")

result = model.generate_content(
    prompt,
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema = Outline.model_schema_json()),
    request_options={"timeout": 600},
)

AttributeError: model_schema_json

In [ ]:
model = ChatGoogleGenerativeAI()

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Outline)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

'{\n  "$defs": {\n    "AlternativeTherapiesSection": {\n      "properties": {\n        "heading": {\n          "default": "Alternative Therapies",\n          "description": "Heading of the section.",\n          "title": "Heading",\n          "type": "string"\n        },\n        "content": {\n          "description": "Summary of non-conventional approaches.",\n          "title": "Content",\n          "type": "string"\n        }\n      },\n      "required": [\n        "content"\n      ],\n      "title": "AlternativeTherapiesSection",\n      "type": "object"\n    },\n    "CausesSection": {\n      "properties": {\n        "heading": {\n          "default": "Causes",\n          "description": "Heading of the section.",\n          "title": "Heading",\n          "type": "string"\n        },\n        "content": {\n          "description": "Explanation of underlying mechanisms or causes, including primary causes.",\n          "title": "Content",\n          "type": "string"\n        }\n      },

In [3]:
Outline.model_json_schema()

{'$defs': {'AlternativeTherapiesSection': {'properties': {'heading': {'default': 'Alternative Therapies',
     'description': 'Heading of the section.',
     'title': 'Heading',
     'type': 'string'},
    'content': {'description': 'Summary of non-conventional approaches.',
     'title': 'Content',
     'type': 'string'}},
   'required': ['content'],
   'title': 'AlternativeTherapiesSection',
   'type': 'object'},
  'CausesSection': {'properties': {'heading': {'default': 'Causes',
     'description': 'Heading of the section.',
     'title': 'Heading',
     'type': 'string'},
    'content': {'description': 'Explanation of underlying mechanisms or causes, including primary causes.',
     'title': 'Content',
     'type': 'string'}},
   'required': ['content'],
   'title': 'CausesSection',
   'type': 'object'},
  'ComplicationsSection': {'properties': {'heading': {'default': 'Complications',
     'description': 'Heading of the section.',
     'title': 'Heading',
     'type': 'string'},
  

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI
from pydantic import BaseModel, Field, model_validator

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)


# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values: dict) -> dict:
        setup = values.get("setup")
        if setup and setup[-1] != "?":
            raise ValueError("Badly formed question!")
        return values


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model
output = prompt_and_model.invoke({"query": "Tell me a joke."})
parser.invoke(output)

In [32]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field, model_validator

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)


prompt_template = """You are a professional scientific writer tasked with developing a detailed and informative knowledgebase article outline on a given condition.

Condition: '{condition}'
Alternate Name: '{alternative_name}'
Category: '{category}'

Detailed Instructions for Each Section:

- **Title**: The main heading of the article, which is the condition itself (e.g., "Diabetes").

- **Subtitle**: A concise introductory phrase summarizing the condition.

- **Sections**: Each section in the article is structured as a key-value pair inside an array. Each element in the array must contain a `"heading"` and `"content"` entry.

    - **Heading**: A string providing the title of the heading. Ensure that all of these headings are included: `"Overview"`, `"Key Facts"`, `"Symptoms"`, `"Types"`, `"Causes"`, `"Risk Factors"`, `"Diagnosis"`, `"Prevention"`, `"Specialist to Visit"`, `"Treatment"`, `"Home-Care"`, `"Living With"`, `"Complications"`, `"Alternative Therapies"`, `"FAQs"`, `"References"`.

    - **Content**: The content for the associated heading. You must adhere to the following requirements:

        - Maintain a professional yet approachable tone.
        - Include hypertext links to relevant sources where appropriate, and format them in the `"References"`, in the same way as a traditional scientific paper.
        - Where possible, include statistics, research findings, or notable insights to make the article credible and informative.
        - Use bullet points where specified to ensure that information is captured in the output.
        - Ensure that content is well-written and contains a brief but sufficient summary.
        - Where subtypes exist, include nested subheadings within the `content` using the `###` markdown header format.

        - **FAQs**: The `content` for the `FAQs` section should be a JSON array of question-answer pairs. Each pair should have a "question" field and an "answer" field. The questions and answers should be concise, clear, and relevant to the condition. Generate 3 to 5 FAQs.

---

Output Requirements:

- Return only the refined JSON object.
- Do not add any extraneous commentary, reasoning steps, or notes outside the JSON object.
- Ensure that all integrated references are formatted in an APA-like style and included in the `"References"` section.
"""


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Outline)

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["condition", "alternative_name", "category"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# And a query intended to prompt a language model to populate the data structure.
prompt_and_model = prompt | model

input_data = {
    "condition": "Diabetes",
    "alternative_name": "Hyperglycemia",
    "category": "Metabolic Disorders"
}

# Invoke the model with the input data
output = prompt_and_model.invoke(input_data)
parser.invoke(output)

OutputParserException: Failed to parse Outline from completion {"Title": "Diabetes", "Subtitle": "A group of metabolic disorders characterized by elevated blood sugar levels.", "Sections": [{"heading": "Overview", "content": "Diabetes is a chronic metabolic disorder affecting how the body regulates blood glucose (sugar).  Over time, high blood sugar can damage organs and lead to serious health complications.  It is a significant global health concern, affecting millions worldwide."}, {"heading": "Key Facts", "content": "- According to the World Health Organization (WHO), diabetes affected an estimated 422 million people worldwide in 2014. \n- There are three main types of diabetes: type 1, type 2, and gestational diabetes. \n- Common symptoms include increased thirst, frequent urination, unexplained weight loss, and blurred vision."}, {"heading": "Symptoms", "content": "Common symptoms include:\n- Increased thirst\n- Frequent urination\n- Unexplained weight loss\n- Increased hunger\n- 

In [33]:
output

AIMessage(content='```json\n{\n  "Title": "Diabetes",\n  "Subtitle": "A group of metabolic disorders characterized by elevated blood sugar levels.",\n  "Sections": [\n    {\n      "heading": "Overview",\n      "content": "Diabetes is a chronic metabolic disorder affecting how the body regulates blood glucose (sugar).  Over time, high blood sugar can damage organs and lead to serious health complications.  It is a significant global health concern, affecting millions worldwide."\n    },\n    {\n      "heading": "Key Facts",\n      "content": "- According to the World Health Organization (WHO), diabetes affected an estimated 422 million people worldwide in 2014. \\n- There are three main types of diabetes: type 1, type 2, and gestational diabetes. \\n- Common symptoms include increased thirst, frequent urination, unexplained weight loss, and blurred vision."\n    },\n    {\n      "heading": "Symptoms",\n      "content": "Common symptoms include:\\n- Increased thirst\\n- Frequent urinatio

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field, ValidationError
from typing import List

# Define SearchQuery, ReferenceItem, FAQItem as before
class SearchQuery(BaseModel):
    section: str = Field(..., description="The section of the outline the query corresponds to.")
    query: str = Field(..., description="The search query for this section.")

class ReferenceItem(BaseModel):
    reference_number: int = Field(..., description="Sequential reference number.")
    authors: str = Field(..., description="Author(s) of the reference.")
    year: str = Field(..., description="Publication year.")
    title: str = Field(..., description="Title of the reference.")
    journal_source: str = Field(..., description="Journal or source of publication.")
    url_doi: str = Field(..., description="URL or DOI of the reference.")

class FAQItem(BaseModel):
    question: str = Field(..., description="A frequently asked question about the condition.")
    answer: str = Field(..., description="A concise and informative answer to the question.")

# Define all section classes without using Union
class Overview(BaseModel):
    heading: str = Field(default="Overview", description="Heading of the Overview section.")
    content: str = Field(..., description="A high-level introduction to the topic, explaining its significance and impact, key statistics, and facts.")

class KeyFacts(BaseModel):
    heading: str = Field(default="Key Facts", description="Heading of the Key Facts section.")
    content: List[str] = Field(..., description="Notable statistics or key data, presented as a list of strings.")

class Symptoms(BaseModel):
    heading: str = Field(default="Symptoms", description="Heading of the Symptoms section.")
    content: List[str] = Field(..., description="Common signs and symptoms, presented as a list of strings.")

class Types(BaseModel):
    heading: str = Field(default="Types", description="Heading of the Types section.")
    content: str = Field(..., description="Classifications or subtypes, each with a brief explanation. Types and subtypes may use nested subheadings (###, ####).")

class Causes(BaseModel):
    heading: str = Field(default="Causes", description="Heading of the Causes section.")
    content: str = Field(..., description="Explanation of underlying mechanisms or causes, including primary causes.")

class RiskFactors(BaseModel):
    heading: str = Field(default="Risk Factors", description="Heading of the Risk Factors section.")
    content: List[str] = Field(..., description="Factors that increase susceptibility, including lifestyle, genetic, or environmental risk factors.")

class Diagnosis(BaseModel):
    heading: str = Field(default="Diagnosis", description="Heading of the Diagnosis section.")
    content: str = Field(..., description="Outline of the diagnostic process, including medical history, clinical symptoms, tests, or imaging tools. Can include subheadings (###) for specific methods.")

class Prevention(BaseModel):
    heading: str = Field(default="Prevention", description="Heading of the Prevention section.")
    content: List[str] = Field(..., description="Practical advice for risk reduction, with evidence-based recommendations.")

class SpecialistToVisit(BaseModel):
    heading: str = Field(default="Specialist to Visit", description="Heading of the Specialist to Visit section.")
    content: str = Field(..., description="Healthcare providers involved in diagnosis and treatment, and their roles.")

class Treatment(BaseModel):
    heading: str = Field(default="Treatment", description="Heading of the Treatment section.")
    content: str = Field(..., description="Description of medical and therapeutic interventions.")

class HomeCare(BaseModel):
    heading: str = Field(default="Home-Care", description="Heading of the Home Care section.")
    content: List[str] = Field(..., description="Tips for self-management, such as lifestyle adjustments, routines, or home remedies.")

class LivingWith(BaseModel):
    heading: str = Field(default="Living With", description="Heading of the Living With section.")
    content: str = Field(..., description="Guidance for long-term management, including emotional, social, or physical adaptation strategies.")

class Complications(BaseModel):
    heading: str = Field(default="Complications", description="Heading of the Complications section.")
    content: str = Field(..., description="Discussion of potential health challenges if the condition is untreated or poorly managed.")

class AlternativeTherapies(BaseModel):
    heading: str = Field(default="Alternative Therapies", description="Heading of the Alternative Therapies section.")
    content: str = Field(..., description="Summary of non-conventional approaches.")

class FAQs(BaseModel):
    heading: str = Field(default="FAQs", description="Heading of the FAQs section.")
    content: List[FAQItem] = Field(..., description="A list of frequently asked questions and their corresponding answers.")

class References(BaseModel):
    heading: str = Field(default="References", description="Heading of the References section.")
    content: List[ReferenceItem] = Field(..., description="A list of references.")

# Refactored Outline class without Union
class Outline(BaseModel):
    title: str = Field(..., description="The main heading of the article.")
    subtitle: str = Field(..., description="A concise introductory phrase summarizing the condition.")
    
    overview: Overview = Field(..., description="Overview section of the article.")
    key_facts: KeyFacts = Field(..., description="Key Facts section of the article.")
    symptoms: Symptoms = Field(..., description="Symptoms section of the article.")
    types: Types = Field(..., description="Types section of the article.")
    causes: Causes = Field(..., description="Causes section of the article.")
    risk_factors: RiskFactors = Field(..., description="Risk Factors section of the article.")
    diagnosis: Diagnosis = Field(..., description="Diagnosis section of the article.")
    prevention: Prevention = Field(..., description="Prevention section of the article.")
    specialist_to_visit: SpecialistToVisit = Field(..., description="Specialist to Visit section of the article.")
    treatment: Treatment = Field(..., description="Treatment section of the article.")
    home_care: HomeCare = Field(..., description="Home-Care section of the article.")
    living_with: LivingWith = Field(..., description="Living With section of the article.")
    complications: Complications = Field(..., description="Complications section of the article.")
    alternative_therapies: AlternativeTherapies = Field(..., description="Alternative Therapies section of the article.")
    faqs: FAQs = Field(..., description="FAQs section of the article.")
    references: References = Field(..., description="References section of the article.")

# Initialize the PydanticOutputParser with the Outline model
parser = PydanticOutputParser(pydantic_object=Outline)

# Create the prompt template with format instructions
prompt_template = PromptTemplate(
    template="""You are a professional scientific writer tasked with developing a detailed and informative knowledgebase article outline on a given condition.

Condition: '{condition}'
Alternate Name: '{alternative_name}'
Category: '{category}'

Detailed Instructions for Each Section:

- **Title**: The main heading of the article, which is the condition itself (e.g., "Diabetes").

- **Subtitle**: A concise introductory phrase summarizing the condition.

- **Sections**: Each section in the article is structured with a specific heading and content. Ensure that all of these sections are included:

    - **Overview**
    - **Key Facts**
    - **Symptoms**
    - **Types**
    - **Causes**
    - **Risk Factors**
    - **Diagnosis**
    - **Prevention**
    - **Specialist to Visit**
    - **Treatment**
    - **Home-Care**
    - **Living With**
    - **Complications**
    - **Alternative Therapies**
    - **FAQs**
    - **References**

For each section, provide the following:

- **Heading**: The title of the section (e.g., "Overview").

- **Content**: Detailed information relevant to the section. Follow these guidelines:

    - Maintain a professional yet approachable tone.
    - Include hypertext links to relevant sources where appropriate, and format them in the "References" section in an APA-like style.
    - Where possible, include statistics, research findings, or notable insights to make the article credible and informative.
    - Use bullet points where specified to ensure that information is captured in the output.
    - Ensure that content is well-written and contains a brief but sufficient summary.
    - Where subtypes exist, include nested subheadings within the content using the `###` markdown header format.

    - **FAQs**: The content for the FAQs section should be a JSON array of question-answer pairs. Each pair should have a "question" field and an "answer" field. Generate 3 to 5 FAQs.

---
{format_instructions}
""",
    input_variables=["condition", "alternative_name", "category"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Initialize the generative model
# model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.0)



model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)

# Combine the prompt and model using the pipe operator
# Note: Ensure that LangChain supports the pipe operator (|) for chaining
prompt_and_model = prompt_template | model

# Define the input data
input_data = {
    "condition": "Diabetes",
    "alternative_name": "Hyperglycemia",
    "category": "Metabolic Disorders"
}

# Invoke the model with the input data
output = prompt_and_model.invoke(input_data)

# Parse and validate the generated content
try:
    outline = parser.invoke(output)
    # Print the validated and formatted JSON
    print(outline.json(indent=4))
except ValidationError as e:
    print("Validation Error:", e)
except Exception as ex:
    print("An error occurred:", ex)


An error occurred: `dumps_kwargs` keyword arguments are no longer supported.


/var/folders/dv/lndb9rrs2kv2v5g8cyfyx0wr0000gn/T/ipykernel_12961/2119195327.py:192: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(outline.json(indent=4))


In [40]:
outline = parser.invoke(output)

print(outline.json(indent=4))

/var/folders/dv/lndb9rrs2kv2v5g8cyfyx0wr0000gn/T/ipykernel_12961/1197983781.py:3: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(outline.json(indent=4))


TypeError: `dumps_kwargs` keyword arguments are no longer supported.